# Python workbook for the graphs for the DoSI publication, taking the results from the analysis without rerunning it.

In [6]:
# Imports and reading

import pandas as pd
import numpy as np
import textwrap

import re
from sklearn.linear_model import LinearRegression
from scipy.stats import linregress
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import nbformat

import plotly.express as px
import plotly.graph_objects as go

VERSION_FOR_DATA = "v26"
VERSION_FOR_FITPARAMETERS = "v27"
VERSION_FOR_METADATA = "v25"
YEAR_PADDING_FOR_PLOTTING = 10

PATH = "/mnt/c/Users/simon.destercke/Documents/misc/iiasa/DoSI"
fn_data = f"{PATH}/adjusted_datasets_{VERSION_FOR_DATA}.csv"
fn_summary = f"""{PATH}/summary_table_{VERSION_FOR_FITPARAMETERS}.csv"""
fn_clusters = f"""{PATH}/PosTip_Clusters.csv""" # Summary file by Charlie
fn_early = f"""{PATH}/EarlyAdopterRegions_perInnovation_21March.csv""" # Early Adopting regions
fn_metadata = f"{PATH}/metadata_master_{VERSION_FOR_METADATA}.xlsx"

dosi_df = pd.read_csv(fn_data, converters={"Indicator Number": str})
dosi_df["Value"] = pd.to_numeric(dosi_df["Value"], errors="coerce")
dosi_df = dosi_df.dropna(subset=["Value"])

# Correct for trailing spaces in the data
dosi_df["Spatial Scale"] = dosi_df["Spatial Scale"].str.rstrip()
dosi_df["Innovation Name"] = dosi_df["Innovation Name"].str.rstrip()

summary_df = pd.read_csv(fn_summary, converters={"Indicator Number": str})

early_df = pd.read_csv(fn_early, usecols = [0,1])
early_dict = dict(zip(early_df.iloc[:, 0], early_df.iloc[:, 1]))


In [7]:
# Metadata / codes

def convert_to_three_digit_notation(s):
    return re.sub(r"([a-zA-Z])(\d+)", lambda m: f"{m.group(1)}{int(m.group(2)):03}", s)


def read_metadata_table(fn, columns):
    df = pd.read_excel(fn, usecols=columns, dtype=str).dropna().reset_index(drop=True)
    df.iloc[:, 1] = df.iloc[:, 1].apply(convert_to_three_digit_notation)
    return df.set_index(df.columns[0])[df.columns[1]].to_dict()

metadata = dict()
metadata["Innovation Name"] = read_metadata_table(fn_metadata, "A,D")
metadata["Spatial Scale"] = read_metadata_table(fn_metadata, "G,I")
metadata["Indicator Number"] = read_metadata_table(
    fn_metadata, "L,O"
)  # Column M is the indicator name. Superfluous because maps 1-1 on indicator number
metadata["Description"] = read_metadata_table(fn_metadata, "R,S")
metadata["Metric"] = read_metadata_table(fn_metadata, "V,W")

for key, nested_dict in metadata.items():
    if isinstance(nested_dict, dict):  # Ensure the value is a dictionary
        metadata[key] = {
            k.lower() if isinstance(k, str) else k: v for k, v in nested_dict.items()
        }

In [8]:
metadata.keys()

dict_keys(['Innovation Name', 'Spatial Scale', 'Indicator Number', 'Description', 'Metric'])

In [9]:
# Attach codes to data file

dosi_df["Innovation Code"] = dosi_df["Innovation Name"].str.lower().map(metadata["Innovation Name"])
dosi_df["Region Code"] = dosi_df["Spatial Scale"].str.lower().map(metadata["Spatial Scale"])
dosi_df["Early Adopter Code"] = dosi_df["Innovation Code"].map(early_dict)
dosi_df["Indicator Code"] = dosi_df["Indicator Number"].str.lower().map(metadata["Indicator Number"])
dosi_df["Description Code"] = dosi_df["Description"].str.lower().map(metadata["Description"])
dosi_df["Metric Code"] = dosi_df["Metric"].str.lower().map(metadata["Metric"])
dosi_df["Code"] = dosi_df[['Innovation Code', 'Region Code', 'Indicator Code', 'Description Code', 'Metric Code']].agg('_'.join, axis=1)

In [10]:
def FPLogValue_with_scaling(x, t0, Dt, s):
    """
    Logistic function with vertical scaling.|
    """
    return s / (1 + np.exp(-np.log(81) * (x - t0) / Dt))

In [11]:
category_abbreviations_dict = {
    "b": "Behavioral",
    "m": "Business Models",
    "s": "Institutional",
}

In [32]:
innovations_df = dosi_df[(dosi_df["Indicator Number"] == '1.1') &
                                (dosi_df["Region Code"] == dosi_df["Early Adopter Code"]) &
                                (
                                    ( (~ dosi_df["Innovation Code"].isin(["pas","tel"])) &
                                    (dosi_df["Metric"] == "market share")
                                ) |
                                ( (dosi_df["Innovation Code"].isin(["pas","tel"])) &
                                    (dosi_df["Metric"].isin(["cumulative # of new units","Partial up to 2019 Percentage of total employment"]))
                                )
                                    )].copy()
innovations_summary_df = summary_df[(summary_df["Code"].str.split('_').str[1] == summary_df["Code"].str.split('_').str[0].map(early_dict)) &
                                            (summary_df["Indicator Number"] == "1.1")].copy()
innovations_summary_df["Category_letter"] = innovations_summary_df["Category"].str.extract(r"^([A-Za-z]+)").iloc[:, 0]
innovations_summary_df["Category_full"] = innovations_summary_df["Category_letter"].map(category_abbreviations_dict)
innovations_summary_df["Innovation Code"] = innovations_summary_df["Code"].str.split('_').str[0]
innovations_summary_df = innovations_summary_df[
                                (
                                    ( (~ innovations_summary_df["Innovation Code"].isin(["pas","tel"])) &
                                    (innovations_summary_df["Metric"] == "market share")
                                ) |
                                ( (innovations_summary_df["Innovation Code"].isin(["pas","tel"])) &
                                    (innovations_summary_df["Metric"].isin(["cumulative # of new units","Partial up to 2019 Percentage of total employment"]))
                                )
                                    )]

In [33]:
innovations_summary_df

,Code,Innovation Name,Spatial Scale,Indicator Number,Indicator Name,Description,Metric,Category,slope_log,slope_exp,...,use_linfit_FIN,select_1.1_earegion_FIN,select_1.1_allregions_FIN,select_non1.1s_earegion_FIN,select_non1.1s_allregions_FIN,Delete from working file,Unnamed: 89,Category_letter,Category_full,Innovation Code
22,act_net_1.1Ado_d332_m185,active mobility,The Netherlands,1.1,Adoption over time,% trips by walking and biking,market share,b7,0.070443,0.026144,...,1.0,1.0,1.0,0.0,0.0,NaN,NaN,b,Behavioral,act
59,cli_swe_1.1Ado_d346_m185,climate protest,Sweden,1.1,Adoption over Time,cumulative share of population participating i...,market share,s3,2.665900,0.662160,...,1.0,1.0,1.0,0.0,0.0,NaN,NaN,s,Institutional,cli
76,coh_den_1.1Ado_d335_m185,co-housing,Denmark,1.1,Adoption over time,share of population living in co-housing projects,market share,b11,0.259730,0.031685,...,1.0,1.0,1.0,0.0,0.0,NaN,NaN,b,Behavioral,coh
102,cro_hei_1.1Ado_d330_m185,car ownership,Heidelberg,1.1,Adaption over time,cars per person,market share,b5,-0.010707,-0.006389,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,b,Behavioral,cro
104,crs_ger_1.1Ado_d340_m185,car sharing,Germany,1.1,Adoption over time,share of drivers who car share,market share,m9,0.264400,0.188820,...,1.0,1.0,1.0,0.0,0.0,NaN,NaN,m,Business Models,crs
153,crz_fra_1.1Ado_d328_m185,mobesity,France,1.1,Adoption over Time,Weight of all new car sales as a share of heav...,market share,b3,0.005642,0.003125,...,1.0,1.0,1.0,0.0,0.0,NaN,NaN,b,Behavioral,crz
207,dig_swe_1.1Ado_d344_m185,digital skills,Sweden,1.1,Adoption over time,share of people engaged in 6 online activities,market share,s1,0.063191,0.027130,...,1.0,1.0,1.0,0.0,0.0,NaN,NaN,s,Institutional,dig
211,dow_swi_1.1Ado_d191_m185,downsizing,Switzerland,1.1,Adoption over time,share of people living in a small dwelling wit...,market share,b10,0.209890,0.162560,...,1.0,1.0,1.0,0.0,0.0,NaN,NaN,b,Behavioral,dow
222,dri_swe_1.1Ado_d334_m185,drivers licence,Sweden,1.1,Adoption over Time,share of teenagers with drivers licenses,market share,b9,-0.038284,-0.022396,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,b,Behavioral,dri
264,eat_ger_1.1Ado_d326_m185,eating less meat,Germany,1.1,Adoption over time,red meat as a share of meat consumption,market share,b1,-0.013897,0.000752,...,1.0,1.0,1.0,0.0,0.0,NaN,NaN,b,Behavioral,eat


In [34]:
# dictionary: category -> list of Innovation Codes
cat_dict = (
    innovations_summary_df.groupby("Category_letter")["Innovation Code"]
      .apply(list)
      .to_dict()
)

# number of categories
n_categories = len(cat_dict)

# length of the longest list
n_within_categories_max = max(len(codes) for codes in cat_dict.values())

# cat_dict, n_categories, n_within_categories_max

In [35]:
cat_dict

{'b': ['act',
  'coh',
  'cro',
  'crz',
  'dow',
  'dri',
  'eat',
  'ebi',
  'low',
  'org',
  'tel'],
 'm': ['crs', 'eco', 'fir', 'foo', 'mic', 'pas', 'sol', 'sus', 'tex'],
 's': ['cli', 'dig', 'ego', 'ene', 'non', 'qui']}

In [36]:
year_min = innovations_df["Year"].min() - YEAR_PADDING_FOR_PLOTTING
year_max = innovations_df["Year"].max() + YEAR_PADDING_FOR_PLOTTING
year_max = 2025 # Override so as not to give the impression that we forecast

years_for_plotting = np.linspace(year_min,year_max, 100)

In [37]:
def plot_line_scatter(ax, x_line, y_line, x_scatter, y_scatter,
                      title=None, xlabel=None, ylabel=None, color = "black"):
    """
    Plot a line and scatter on an existing Axes.
    """
    if x_line is not None and y_line is not None:
        ax.plot(x_line, y_line, lw=1, label="line", color=color, alpha=1)
    if x_scatter is not None and y_scatter is not None:
        ax.scatter(x_scatter, y_scatter, s=3, label="scatter", alpha=0.7, color = color)

    if title:
        ax.set_title(title, fontsize=9, pad=4)
    if xlabel:
        ax.set_xlabel(xlabel)
    if ylabel:
        ax.set_ylabel(ylabel)

    ax.spines["right"].set_visible(False)
    ax.spines["top"].set_visible(False)
    ax.tick_params(labelsize=6)
    ax.tick_params(axis='y', pad=1)
    ax.set_ylim(0,1.2)

In [38]:
def add_info_box(ax, lines, padding_top = 0.03, padding_left = 0.03, fontsize=10):
    """
    Add a small annotation box with black outline and white background
    in the top-left corner of a matplotlib Axes.

    Parameters
    ----------
    ax : matplotlib.axes.Axes
        The axes to draw the box on.
    lines : list of str
        The text lines to display (e.g. ["x = 1.234", "Δx = 0.0567", "x = 2.3456"]).
    padding : float
        Fraction of the axes width/height to pad from top-left.
    fontsize : int
        Font size of the text inside the box.
    """
    # Join the lines into one block of text
    textstr = "\n".join(lines)

    # Place the box in the top left
    ax.text(
        padding_left, 1 - padding_top, textstr,
        transform=ax.transAxes,
        fontsize=fontsize,
        verticalalignment='top',
        horizontalalignment='left',
        bbox=dict(
            boxstyle="round,pad=0.3",
            facecolor="white",
            edgecolor=None,
            alpha=0,
        )
    )

In [39]:
colors = [
    "#000000",  # black
    "#E69F00",  # orange
    "#56B4E9",  # sky blue
    "#009E73",  # bluish green
    "#0072B2",  # blue
    "#D55E00",  # vermillion
    "#CC79A7",  # purple
    "#666666",  # dark gray
    "#999999",  # light gray
    "#8C510A",  # brown
    "#1B9E77"   # teal
]


In [40]:
innovations_to_exclude = ['crz', # mobesity
                          'low', # low-carbon long distance travel
                          'cli', # climate protests
                          ]

In [ ]:
fig, axes = plt.subplots(n_categories, n_within_categories_max,
                         figsize=(15, 10),
                         squeeze=False)

for i, category in enumerate(cat_dict.keys()):

    shifter = 0

    for j in range(n_within_categories_max):    
        
        if j >= len(cat_dict[category]) - shifter:
            # If there are fewer innovations than the max, fill with empty plots
            axes[i, j].axis("off")
            axes[i, j].set_visible(False)
            continue

        else: 

            if cat_dict[category][j] in innovations_to_exclude:
                shifter += 1

            innovation_code = cat_dict[category][j+shifter]

            ax = axes[i, j]

            t0 = innovations_summary_df[innovations_summary_df["Innovation Code"] == innovation_code]["log_t0"].iloc[0]
            Dt = innovations_summary_df[innovations_summary_df["Innovation Code"] == innovation_code]["log_Dt"].iloc[0]
            K = innovations_summary_df[innovations_summary_df["Innovation Code"] == innovation_code]["log_K"].iloc[0]

            title = f"""{next(key for key, value in metadata["Innovation Name"].items() if value == innovation_code).title()} ({next(key for key, value in metadata["Spatial Scale"].items() if value == early_dict[innovation_code]).title()})"""
            wrapped_title = "\n".join(textwrap.wrap(title, width=18))

            # call plotting function
            plot_line_scatter(ax, years_for_plotting, FPLogValue_with_scaling(years_for_plotting, t0, Dt, K)/K, innovations_df[innovations_df["Innovation Code"] == innovation_code]["Year"], innovations_df[innovations_df["Innovation Code"] == innovation_code]["Value"]/K,
                                title=wrapped_title, color = "black",
                                xlabel="Year", ylabel=None)
            
            add_info_box(ax, [
                f"t$_0$ = {t0:.0f}",
                f"Δt = {Dt:.0f}y",
                f"K = {K:.2g}",
                f"n = {len(innovations_df[innovations_df['Innovation Code'] == innovation_code])}"
            ], padding_top=0.02, padding_left = 0.01 if Dt > 0 else .6, fontsize=5)

            ax.tick_params(axis='y', pad=0)
            
            # Rotate the existing tick labels
            for label in ax.get_xticklabels():
                label.set_rotation(45)
                label.set_rotation_mode("anchor")  # <-- critical
                label.set_ha("right")              # tick mark aligns with right edge of text
                label.set_va("center")             # vertical center crosses tick

            # add row label on the left
            if j == 0:
                ax.set_ylabel(category_abbreviations_dict[category], fontsize=9, labelpad=10)

fig.tight_layout()
plt.show()

fig.savefig(f"{PATH}/dosi_publication_graphs_fig1_20250901_Knormalized.pdf", bbox_inches="tight", dpi=300)

In [ ]:
# Combine columns
fig, axes = plt.subplots(n_categories, 1,
                         figsize=(5, 8),
                         squeeze=False)


for i, category in enumerate(cat_dict.keys()):
    for j, innovation_code in enumerate(cat_dict[category]):

        if innovation_code in innovations_to_exclude:
            continue   

        else:
        
            ax = axes[i, 0]

            t0 = innovations_summary_df[innovations_summary_df["Innovation Code"] == innovation_code]["log_t0"].iloc[0]
            Dt = innovations_summary_df[innovations_summary_df["Innovation Code"] == innovation_code]["log_Dt"].iloc[0]
            K = innovations_summary_df[innovations_summary_df["Innovation Code"] == innovation_code]["log_K"].iloc[0]

            title = f"""{next(key for key, value in metadata["Innovation Name"].items() if value == innovation_code).title()} ({next(key for key, value in metadata["Spatial Scale"].items() if value == early_dict[innovation_code]).title()})"""
            wrapped_title = "\n".join(textwrap.wrap(title, width=18))

            # call plotting function
            plot_line_scatter(ax, years_for_plotting, (1/K if Dt < 0 else 0) + (-1 if Dt < 0 else 1) * FPLogValue_with_scaling(years_for_plotting, t0, Dt, K)/K, innovations_df[innovations_df["Innovation Code"] == innovation_code]["Year"], (1/K if Dt < 0 else 0) + (-1 if Dt < 0 else 1) * innovations_df[innovations_df["Innovation Code"] == innovation_code]["Value"]/K,
                                title=None,
                                xlabel="Year", ylabel=None, color = colors[j % len(colors)])
            
            # Rotate the existing tick labels
            for label in ax.get_xticklabels():
                label.set_rotation(45)
                label.set_rotation_mode("anchor")  # <-- critical
                label.set_ha("right")              # tick mark aligns with right edge of text
                label.set_va("center")             # vertical center crosses tick

            # add row label on the left
            if j == 0:
                ax.set_ylabel(category_abbreviations_dict[category], fontsize=9, labelpad=10)

            x_centroid = innovations_df[innovations_df["Innovation Code"] == innovation_code]["Year"].median()
            y_centroid = ((1/K if Dt < 0 else 0) + (-1 if Dt < 0 else 1) * (innovations_df[innovations_df["Innovation Code"] == innovation_code]["Value"]/K)).median()
            first_x = innovations_df[innovations_df["Innovation Code"] == innovation_code]["Year"].min()
            first_y = ((1/K if Dt < 0 else 0) + (-1 if Dt < 0 else 1) * innovations_df[innovations_df["Innovation Code"] == innovation_code]["Value"].iloc[0]/K)

            ax. annotate(next(key for key, value in metadata["Innovation Name"].items() if value == innovation_code).title(),
                        xy=(x_centroid, y_centroid),
                        #xy =(first_x, first_y),
                        color = colors[j % len(colors)], size = 5,   ha="center",      va="center"    )


fig.tight_layout()
plt.show()

fig.savefig(f"{PATH}/dosi_publication_graphs_fig1v2_Knormalized_20250901.pdf", bbox_inches="tight", dpi=300)